In [ ]:
from multiprocessing import Pool
import requests
import pandas as pd
import time
from ratelimit import limits, sleep_and_retry

mesAno = [202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112]

codigoIbge = [1400100,1400605,1400506,1400233,1400704,1400407,1400282,1400159,1400027,1400050,1400308,1400175,1400456,1400209,1400472]

dados = []
dados2 = []
results = ["string"]
api_url = "https://api.portaldatransparencia.gov.br/api-de-dados/auxilio-emergencial-beneficiario-por-municipio"
ONE_MINUTE = 60

@sleep_and_retry
@limits(calls=180, period=ONE_MINUTE)
def call_api(api_url):
    response = requests.get(api_url, params=params, headers=headers)

    # if response.status_code != 200:
    #     raise Exception('API response: {}'.format(response.status_code))
    return response

def get_data(mesAno,codigoIbge,pagina=0):

    while len(results) != 0:

        start = time.time()
        pagina += 1
        params = {"mesAno": mesAno, "codigoIbge": codigoIbge,"pagina":pagina}
        headers = {"chave-api-dados": "_7e44a488bb29946885b8d1789810b0ea"}
        #response = requests.get(api_url, params=params, headers=headers)
        call_api(api_url)
        results = response.json()
        flat_data = pd.json_normalize(results)
        dados.append(flat_data)
        stop = time.time()
        print(f"Página {pagina} capturada com sucesso em {stop - start} segundos")

if __name__ == "__main__":
    p = Pool(6)
    start = time.time()
    p.starmap(get_data,zip(mesAno,codigoIbge))
    end = time.time()
    print(f"MP concluído em {end - start}")
    p.close()
    p.join()